## Types of Chat Bot's
In the world of machine learning and AI there are many different kinds of chat bots. Some chat bots are virtual assistants, others are just there to talk to, some are customer support agents and you've probably seen some of the ones used by businesses to answer questions. For this tutorial we will be creating a relatively simple chat bot that will be be used to answer frequently asked questions.

## Install Packages
Before starting to work on our chatbot we need to download a few python packages. Please note as of writing this these packages will ONLY WORK IN PYTHON 3.6. Hopefully this will be fixed in the future.

We will simply use pip to install the following:
- numpy
- nltk
- tensorflow
- tflearn

Simply go to CMD and type: pip install "package name". Where you will replace "package_name" with all of the entries listed above.

### Training Data
Now it's time to understand what kind of data we will need to provide our chatbot with. Since this is a simple chatbot we don't need to download any massive datasets. We will just use data that we write ourselves. To follow along with the tutorial properly you will need to create a .JSON file that contains the same format as the one seen below. I've called my file "intents.json".

In [1]:
#from google.colab import drive
#drive.mount("/content/drive")

In [2]:
!ls '/content/drive/MyDrive/Colab Notebooks/notebooks'

ls: /content/drive/MyDrive/Colab Notebooks/notebooks: No such file or directory


What we are doing with the JSON file is creating a bunch of messages that the user is likely to type in and mapping them to a group of appropriate responses. The tag on each dictionary in the file indicates the group that each message belongs too. With this data we will train a neural network to take a sentence of words and classify it as one of the tags in our file. Then we can simply take a response from those groups and display that to the user. The more tags, responses, and patterns you provide to the chatbot the better and more complex it will be.

### Loading our JSON Data
We will start by importing some modules and loading in our json data. Make sure that your .json file is in the same directory as your python script!

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random

import json
with open('intents.json') as file:
    data = json.load(file)
    
print('done')

Instructions for updating:
non-resource variables are not supported in the long term
done


In [4]:
#!pip install tflearn

In [2]:
import nltk 
#nltk.download('punkt')

## Extracting Data
Now its time to take out the data we want from our JSON file. We need all of the patterns and which class/tag they belong to. We also want a list of all of the unique words in our patterns (we will talk about why later), so lets setup some blank lists to store these values.

words = []
labels = []
docs_x = []
docs_y = []
Now its time to loop through our JSON data and extract the data we want. For each pattern we will turn it into a list of words using nltk.word_tokenizer, rather than having them as strings. We will then add each pattern into our docs_x list and its associated tag into the docs_y list.

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
In the next tutorial we will do some preprocessing of this data and get it ready to feed to our neural network.

In [3]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
print(labels)

['Q0', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'Q75', 'Q76', 'Q77', 'Q78', 'Q79', 'Q80', 'Q81', 'Q82', 'Q83', 'Q84', 'Q85', 'Q86', 'Q87', 'Q88', 'Q89', 'Q90', 'Q91', 'Q92', 'Q93', 'Q94', 'Q95', 'Q96', 'Q97', 'Q98', 'Q99', 'Q100', 'Q101', 'Q102', 'Q103', 'Q104', 'Q105', 'Q106', 'Q107', 'Q108', 'Q109', 'Q110', 'Q111', 'Q112', 'Q113', 'Q114', 'Q115', 'Q116', 'Q117', 'Q118', 'Q119', 'Q120', 'Q121', 'Q122', 'Q123', 'Q124', 'Q125', 'Q126', 'Q127', 'Q128', 'Q129', 'Q130', 'Q131', 'Q132', 'Q133', 'Q134', 'Q135', 'Q136', 'Q137', 'Q138

## Word Stemming
You may have heard me talk about word stemming in the previous tutorial. Stemming a word is attempting to find the root of the word. For example, the word "thats" stem might be "that" and the word "happening" would have the stem of "happen". We will use this process of stemming words to reduce the vocabulary of our model and attempt to find the more general meaning behind sentences.

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

This code will simply create a unique list of stemmed words to use in the next step of our data preprocessing.

## Bag of Words
Now that we have loaded in our data and created a stemmed vocabulary it's time to talk about a bag of words. As we know neural networks and machine learning algorithms require numerical input. So out list of strings wont cut it. We need some way to represent our sentences with numbers and this is where a bag of words comes in. What we are going to do is represent each sentence with a list the length of the amount of words in our models vocabulary. Each position in the list will represent a word from our vocabulary. If the position in the list is a 1 then that will mean that the word exists in our sentence, if it is a 0 then the word is nor present. We call this a bag of words because the order in which the words appear in the sentence is lost, we only know the presence of words in our models vocabulary.

As well as formatting our input we need to format our output to make sense to the neural network. Similarly to a bag of words we will create output lists which are the length of the amount of labels/tags we have in our dataset. Each position in the list will represent one distinct label/tag, a 1 in any of those positions will show which label/tag is represented.

In [4]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)
print(output)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Developing a Model
Now that we have preprocessed all of our data we are ready to start creating and training a model. For our purposes we will use a fairly standard feed-forward neural network with two hidden layers. The goal of our network will be to look at a bag of words and give a class that they belong too (one of our tags from the JSON file).

We will start by defining the architecture of our model. Keep in mind that you can mess with some of the numbers here and try to make an even better model! A lot of machine learning is trial an error.


## Training & Saving the Model
Now that we have setup our model its time to train it on our data! To do these we will fit our data to the model. The number of epochs we set is the amount of times that the model will see the same information while training.

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")
Once we are done training the model we can save it to the file model.tflearn for use in other scripts.

In [8]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model_chat101/model.tflearn")

Training Step: 112999  | total loss: 0.00568 | time: 0.173s
| Adam | epoch: 1000 | loss: 0.00568 - acc: 0.9998 -- iter: 896/903
Training Step: 113000  | total loss: 0.00511 | time: 0.174s
| Adam | epoch: 1000 | loss: 0.00511 - acc: 0.9998 -- iter: 903/903
--
INFO:tensorflow:/Users/micintro/Desktop/ChatBot/Mad-Chatter/Matts Work/Expanded reaserch/Model Evaluation/model_chat101/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Making Predictions
Now its time to actually use the model! Ideally we want to generate a response to any sentence the user types in. To do this we need to remember that our model does not take string input, it takes a bag of words. We also need to realize that our model does not spit out sentences, it generates a list of probabilities for all of our classes. This makes the process to generate a response look like the following:
– Get some input from the user
– Convert it to a bag of words
– Get a prediction from the model
– Find the most probable class
– Pick a response from that class



The bag_of_words function will transform our string input to a bag of words using our created words list. The chat function will handle getting a prediction from the model and grabbing an appropriate response from our JSON file of responses.

Now run the program and enjoy chatting with your bot!

In [5]:
#set up model
tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load('model_chat101/model.tflearn')
    print('model loaded')
except:
    print('no model to load')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from /Users/micintro/Desktop/ChatBot/Mad-Chatter/Matts Work/Expanded reaserch/Model Evaluation/model_chat101/model.tflearn
model loaded


In [6]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


def respond(text:str):
    if(text == None or text == ''):
        return

    results = model.predict([bag_of_words(text, words)])
    results_index = numpy.argmax(results)
    results_score = numpy.amax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']
    if(results_score> 0.6):
        return(random.choice(responses))
    else:
        return('Sorry I do not understand the question, can you please try asking agin.')
        
def question(text):  
    text = text.lower()
    if(('hello' in text or 'hi' in text or 'good day' in text or 'hey' in text) and len(text)<15):
        return 'Hello there what can i help you with today?'
    if(('what are you' in text or 'what is this' in text or 'how do i use' in text or
        'help' in text)and len(text)< 20):
        return 'I am an QA AI assistant for Maximus ask your work related question and I will do my best to help.'
    
    results = model.predict([bag_of_words(text, words)])
    results_index = numpy.argmax(results)
    results_score = numpy.amax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    result = (random.choice(responses))
    if(results_score> 0.95):
        return(result)
    else:
        return('Sorry I do not understand the question, can you please try asking again.')

In [44]:
#run this to chat 
chat()

Start talking with the bot (type quit to stop)!
You: hello
Refer to [SOP TD 1.6 Instructor Observation](https://maximus365.sharepoint.com/sites/CCO/Resources/SOP/Training%20Delivery/Forms/AllItems.aspx?viewpath=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FForms%2FAllItems%2Easpx&id=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FTD%201%2E6%5FInstructor%5FObservation%5Fv4%2E6%2Epdf&parent=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery)
You: exit
Refer to [SOP TD 1.6 Instructor Observation](https://maximus365.sharepoint.com/sites/CCO/Resources/SOP/Training%20Delivery/Forms/AllItems.aspx?viewpath=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FForms%2FAllItems%2Easpx&id=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FTD%201%2E6%5FInstructor%5FObservation%5Fv4%2E6%2Epdf&parent=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery)
You: quit


In [12]:
#run this to send individual data 
question('hello?')

'Refer to [SOP TD 1.6 Instructor Observation](https://maximus365.sharepoint.com/sites/CCO/Resources/SOP/Training%20Delivery/Forms/AllItems.aspx?viewpath=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FForms%2FAllItems%2Easpx&id=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery%2FTD%201%2E6%5FInstructor%5FObservation%5Fv4%2E6%2Epdf&parent=%2Fsites%2FCCO%2FResources%2FSOP%2FTraining%20Delivery)'

In [7]:
import json
  
# Opening JSON file
f = open('intents.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'Q0', 'patterns': ['A CSR in my class cannot access their CCO Learning ', 'A CSR in my class cannot get into their CCO Learning. ', 'A CSR in my class cannot log into their CCO Learning ', 'A participant in my class cannot access their CCO Learning ', 'A participant in my class cannot get into their CCO Learning. ', 'A participant in my class cannot log into their CCO Learning ', 'A trainee in my class cannot access their CCO Learning. ', 'A trainee in my class cannot get into their CCO Learning.  ', 'A trainee in my class cannot log into their CCO Learning. ', 'How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ', 'Who do I reach out to if a trainee cannot access CCO Learning?'], 'responses': ['Refer to [IT Service Desk](https://itservicedesk.maximus.com/CherwellPortal/IT?_=422bffd2) complete and submit a ticket reporting CCO Learning issue. '], 'context_set': ''}
{'tag': 'Q1', 'patterns': ['A CSR in my class locked themselves out of their app

In [8]:
questions =[]
answers = []

for i in data['intents']:
    questions.append(i['patterns'])
    answers.append(i['responses'][0])
    
print(questions)
print('')
print('')
print(answers)

[['A CSR in my class cannot access their CCO Learning ', 'A CSR in my class cannot get into their CCO Learning. ', 'A CSR in my class cannot log into their CCO Learning ', 'A participant in my class cannot access their CCO Learning ', 'A participant in my class cannot get into their CCO Learning. ', 'A participant in my class cannot log into their CCO Learning ', 'A trainee in my class cannot access their CCO Learning. ', 'A trainee in my class cannot get into their CCO Learning.  ', 'A trainee in my class cannot log into their CCO Learning. ', 'How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ', 'Who do I reach out to if a trainee cannot access CCO Learning?'], ['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ', 'How to handle the clear2work app if a trainee gets locked out of the app?', 'One of my new hires is locked out of the Health Assessment application. Who should I notify to 

In [9]:
# set up parrot to do parra phrases 
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
print('parrot loaded ready to work')


parrot loaded ready to work


In [10]:
##Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [11]:
#build compresed method 
def get_para(phrase, num, length):
    try:
        texts = []
        para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False, diversity_ranker='levenshtein', do_diverse=False,
                                    max_return_phrases=num, max_length=length, adequacy_threshold=0.9, fluency_threshold=0.9)
        for para_phrase in para_phrases:
            text = para_phrase[0]
            texts.append(text)

        return texts
    except:
        return[phrase]
        
        

text = 'Whats the most delicious papayas?'
texts = get_para(text, 10, len(text)+5)
print(texts)

["what's the best papaya in the world?", 'tell me the most delicious papaya in the world?', 'show some of the best papayas?', 'can you list the best papayas?', 'can you list the best of all best papayas?', 'which are the best papayas?', 'can you list the most delectable papayas?', 'can you list the most delicious papayas?']


In [ ]:
# # now run all intents and make 0 succsess 1 fail on a hard set
# results = []

# index =0
# for x in questions:
#     answer = answers[index]
#     for i in x:
#         texts = get_para(i, 10, len(text)+5)
#         texts.append(i)
#         print(texts)
#         for text in texts:
#             res = question(text)
#             #print('AI :', res)
#             #print('answer :', answer)
#             if(answer == res):
#                 results.append(0)
#             else:
#                 results.append(1)
#     index +=1
#     print(results)

# print('')
# print('these are the final results')
# print(results)

In [12]:
# now run all intents and make 0 succsess 1 fail on a hard set
results = []
sorry =0
index =0
for x in questions:
    answer = answers[index]
    texts = get_para(x[0], 1, len(text)+5)
    #texts.append(i)
    print(texts)
    for text in texts:
        res = question(text)
        #print('AI :', res)
        #print('answer :', answer)
        if(answer == res or res == 'Sorry I do not understand the question, can you please try asking again.'):
            results.append(0)       
        else:
            results.append(1)
        if(res == 'Sorry I do not understand the question, can you please try asking again.'):
            sorry+=1  

    index +=1
    print(results)

print('')
print('these are the final results')
print(results)
print('please try again count =',sorry)

['A CSR in my class cannot access their CCO Learning ']
[0]
['how does a csr in my class complete the daily health assessment?']
[0, 1]
['A CSR quit but their timecard is wrong.  How can I get their timecard corrected? ']
[0, 1, 0]
['A step-by-step guideline on how to fill in specific sections of the daily training report  on the daily basis.']
[0, 1, 0, 0]
['a trainee has questions about the tax forms']
[0, 1, 0, 0, 0]
['A trainee in my class had their cell phone go off in class, what now?']
[0, 1, 0, 0, 0, 0]
['is my daily training report correctly completed?']
[0, 1, 0, 0, 0, 0, 0]
['can i make calls during the class?']
[0, 1, 0, 0, 0, 0, 0, 0]
['and what is your primary priority for the first day of nesting?']
[0, 1, 0, 0, 0, 0, 0, 0, 1]
['can i copy and load cco learning courses in my training class?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
['what deltek charge codes do we need for days 1 and 2?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['is there a daily schedule i need to complete in a class?

['How do I resolve Windows access issues in my class?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['how can i set up virtual training?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['How do I submit a reset password ticket correctly?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
["How do I submit a ticket to unlock a trainee's computer"]
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['how do we find out which departments to reach out for different issues

['I need a Call Cert form. Where do I find a form?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['i need some ideas for icebreakers']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
['i need some ideas for the reviews activities']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
['is there a sharepoint system in class for social activ

['What form do I fill out for CSR Certification?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
['What happens if I forget to do the Daily Training Report every day?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['What happens if I get a bad Level 1 score?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

['where are my school rosters?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
['where can i find instructor manuals for marketplace?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
['where can i find instructor manuals for medicare?']
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [13]:
# now run all intents and make 0 succsess 1 fail on a medium set
results = []
sorry =0
index =0
for x in questions:
    answer = answers[index]
    texts = get_para(x[0], 1, len(text))
    #texts.append(i)
    print(texts)
    for text in texts:
        res = question(text)
        #print('AI :', res)
        #print('answer :', answer)
        if(answer == res or res == 'Sorry I do not understand the question, can you please try asking again.'):
            results.append(0)       
        else:
            results.append(1)
        if(res == 'Sorry I do not understand the question, can you please try asking again.'):
            sorry+=1  

    index +=1
    print(results)

print('')
print('these are the final results')
print(results)
print('please try again count =',sorry)

["a csr in my class can't access their cco learning "]
[0]
['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ']
[0, 0]
['A CSR quit but their timecard is wrong.  How can I get their timecard corrected? ']
[0, 0, 0]
['A step-by-step guideline on how to fill in specific sections of the daily training report  on the daily basis.']
[0, 0, 0, 0]
['the trainee has questions about the tax forms']
[0, 0, 0, 0, 0]
['when the cell phone goes off in class what happens?']
[0, 0, 0, 0, 0, 1]
['Am I completing Daily Training Report correctly?']
[0, 0, 0, 0, 0, 1, 0]
['is it possible to take phone calls while taking class?']
[0, 0, 0, 0, 0, 1, 0, 1]
['And what is priority for Day 1 Nesting?']
[0, 0, 0, 0, 0, 1, 0, 1, 0]
["what's the best way to add cco learning courses to my training class?"]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
['Are there different deltek charge codes we are supposed to use on days 1 and 2? ']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 

["how do i resolve a windows-based access issue in a teacher's class?"]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
['how can i start to set up virtual training?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
['how can i submit a ticket for a reset of my password?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0]
["How do I submit a ticket to unlock a trainee's computer"]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
['How do we know what department to reach ou

['I need a Call Cert form. Where do I find a form?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]
['i need icebreaker ideas']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
['i need ideas for the review activities']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
['is there a central sharepoint site for socially dinstanced activities?

['what form does a csr certification form form need to fill?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
['What happens if I forget to do the Daily Training Report every day?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
['what happens if a level 1 score is lower than a level 2 score?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['Where are my class rosters?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Where are my instructor guides for Marketplace?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['which are the best books or resources for medicare?']
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
# # now run all intents and make 0 succsess 1 fail on a easy set
# results = []

# index =0
# for x in questions:
#     answer = answers[index]
#     for i in x:
#         texts =[i]
#         print(texts)
#         for text in texts:
#             res = question(text)
#             #print('AI :', res)
#             #print('answer :', answer)
#             if(answer == res):
#                 results.append(0)
#             else:
#                 results.append(1)
#     index +=1
#     print(results)

# print('')
# print('these are the final results')
# print(results)

['A CSR in my class cannot access their CCO Learning ']
['A CSR in my class cannot get into their CCO Learning. ']
['A CSR in my class cannot log into their CCO Learning ']
['A participant in my class cannot access their CCO Learning ']
['A participant in my class cannot get into their CCO Learning. ']
['A participant in my class cannot log into their CCO Learning ']
['A trainee in my class cannot access their CCO Learning. ']
['A trainee in my class cannot get into their CCO Learning.  ']
['A trainee in my class cannot log into their CCO Learning. ']
['How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ']
['Who do I reach out to if a trainee cannot access CCO Learning?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ']
['How to handle the clear2work app if a trainee gets locked out of the app?']
['One of my new hires is locked out of the Health Ass

['What is not allowed to have in a Training Room according the the Secure Floor Policy?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Can I correct an IT ticket after it was submitted?']
['Can I fix an IT ticket after it was submitted? ']
['I submitted a ticket to IT incorrectly. Can I correct it?']
['Is there a way to correct a ticket after it was submitted?']
['What happens if I submit a ticket incorrectly?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

['What are the steps to upload my Daily Training Report?']
['What are the steps to uploading my DTR?']
['What do I name my Daily Training Report?']
['What do I name my DTR?']
['What information should I include on the class DTR?']
['What is expected to be recorded in the Daily Training Report?']
['What is expected to be recorded in the DTR?']
['What is the naming convention for my class Daily Training Report?']
['Who is supposed to fill out the Daily Training Report daily?']
['Who is supposed to fill out the DTR daily?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

['What is the required information to properly request a separation? ']
['What is the required information to properly request a term? ']
['What is the required information to properly request a termination? ']
['What were the proper steps to process a term request EDI form completion etc ']
['What were the proper steps to process a term request EDI form completion?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

["Where do I find my trainee's employee number?"]
["Where do I find my trainee's LAN ID?"]
["Where do I find my trainee's Next Generation Desktop Production ID?"]
["Where do I find my trainee's NGD Production ID?"]
["Where do I find my trainee's production ID?"]
["Where do I find my trainee's username for Windows?"]
['Where is my participant list?']
["Where is new hire's log in information?"]
["Where is trainee's log in information?"]
['Who is assigned to my class?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Save the data to CSV

now we take the results and save them to a csv format 



In [13]:
import csv

header = ['index', 'pred', 'expected']
data = []

index = 0
for x in results:
    item = [index,x,0]
    data.append(item)
    index +=1
    
    

#with open('csv data/chat_101_data.csv', 'w', encoding='UTF8', newline='') as f:
with open('csv data/chat_101_data_hard.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

print('done all data writen to csv')

done all data writen to csv
